# A/B Testing Analysis for Ad Campaign

## Data Source
The data used is from Kaggle, linked [here](https://www.kaggle.com/datasets/amirmotefaker/ab-testing-dataset/data). 

## Goal 
The goal of this notebook is to analyze marketing techniques using A/B testing, comparing a test campaign to a control campaign, utilizing inferential statistics (hypothesis testing) and visualizations. Through this, we should be able to identify if the test campaign made a difference in the amount of customers. 

## Skills Used
* Python Programming
* Data cleaning
* Feature Engineering
* Descriptive Statistics
* Hypothesis Testing (with SciPy)
* Time Series Visualization (with Plotly) 
* Histogram Visualization (with Plotly)

In [115]:
##import dependencies 

import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import sklearn 
from scipy.stats import shapiro 
from scipy.stats import normaltest


### Import Data 
There are two data files to import, one for the control group and one for the test group. Note that for this data, instead of being comma-separated, it is **semi-colon separated**, so when reading the files we have to indicate "sep*** ***= ';'"

In [116]:
control_data = pd.read_csv('data/control_group.csv', sep = ';')
test_data = pd.read_csv('data/test_group.csv', sep = ';')


In [318]:
#Check out the data heads for each: 
control_data.head()

,Campaign,Date,Amount Spent,Impressions,Reach,Num Clicks,Num Searches,Num Views,Num Added to Cart,Num Purchases
0,Control Campaign,2019-01-08,2280,82702.000000,56930.000000,7016.000000,2290.000000,2159.000000,1819.0,618.000000
1,Control Campaign,2019-02-08,1757,121040.000000,102513.000000,8110.000000,2033.000000,1841.000000,1219.0,511.000000
2,Control Campaign,2019-03-08,2343,131711.000000,110862.000000,6508.000000,1737.000000,1549.000000,1134.0,372.000000
3,Control Campaign,2019-04-08,1940,72878.000000,61235.000000,3065.000000,1042.000000,982.000000,1183.0,340.000000
4,Control Campaign,2019-05-08,1835,109559.758621,88844.931034,5320.793103,2221.310345,1943.793103,1300.0,522.793103


In [118]:
test_data.head()

,Campaign Name,Date,Spend [USD],# of Impressions,Reach,# of Website Clicks,# of Searches,# of View Content,# of Add to Cart,# of Purchase
0,Test Campaign,1.08.2019,3008,39550,35820,3038,1946,1069,894,255
1,Test Campaign,2.08.2019,2542,100719,91236,4657,2359,1548,879,677
2,Test Campaign,3.08.2019,2365,70263,45198,7885,2572,2367,1268,578
3,Test Campaign,4.08.2019,2710,78451,25937,4216,2216,1437,566,340
4,Test Campaign,5.08.2019,2297,114295,95138,5863,2106,858,956,768


In [119]:
# Let's see how big these datasets are: 
print(f'Control dataset has a length of: {str(len(control_data))}')
print(f'Test dataset has a length of: {str(len(test_data))}')


Control dataset has a length of: 30
Test dataset has a length of: 30


Pretty small dataset!

### Cleanup 

The column names are a little much, so I will change them to be a little more concise

In [120]:
new_col_names = ['Campaign', 'Date', 'Amount Spent', 'Impressions', 'Reach', 'Num Clicks', 
                'Num Searches', 'Num Views', 'Num Added to Cart', 'Num Purchases']

control_data.columns = new_col_names
test_data.columns = new_col_names 

test_data.head(1)

,Campaign,Date,Amount Spent,Impressions,Reach,Num Clicks,Num Searches,Num Views,Num Added to Cart,Num Purchases
0,Test Campaign,1.08.2019,3008,39550,35820,3038,1946,1069,894,255


In [121]:
control_data.head(1)

,Campaign,Date,Amount Spent,Impressions,Reach,Num Clicks,Num Searches,Num Views,Num Added to Cart,Num Purchases
0,Control Campaign,1.08.2019,2280,82702.0,56930.0,7016.0,2290.0,2159.0,1819.0,618.0


Now I can look at if there are any missing values in the datasets

In [122]:
control_data.isnull().sum()

Campaign             0
Date                 0
Amount Spent         0
Impressions          1
Reach                1
Num Clicks           1
Num Searches         1
Num Views            1
Num Added to Cart    1
Num Purchases        1
dtype: int64

In [123]:
test_data.isnull().sum()

Campaign             0
Date                 0
Amount Spent         0
Impressions          0
Reach                0
Num Clicks           0
Num Searches         0
Num Views            0
Num Added to Cart    0
Num Purchases        0
dtype: int64

Only the control data has missing values

Because the data is already smaller in sample size, instead of dropping the rows with missing values, I will fill the values with the mean of the affected columns

In [124]:
need_to_fill = ['Impressions', 'Reach', 'Num Clicks', 'Num Searches', 'Num Views',
               'Num Added to Cart', 'Num Purchases']

for col in need_to_fill: 
    mean = control_data[col].mean()
    control_data[col].fillna(mean, inplace = True)
    

In [125]:
control_data.isnull().sum()

Campaign             0
Date                 0
Amount Spent         0
Impressions          0
Reach                0
Num Clicks           0
Num Searches         0
Num Views            0
Num Added to Cart    0
Num Purchases        0
dtype: int64

In our datasets,we have some date values. Let's see what form they are in: 

In [126]:
print(control_data['Date'].dtype)
print(test_data['Date'].dtype)


object
object


To make things easier later on when I do visualizations, I will convert the datatype of the Date columns into datetime

In [127]:
control_data['Date'] = pd.to_datetime(control_data['Date'])
test_data['Date'] = pd.to_datetime(test_data['Date'])

print(control_data['Date'].dtype)
print(test_data['Date'].dtype)

datetime64[ns]
datetime64[ns]


### Descriptive Stats

We can get a quick overview of the descriptive stats for each dataset: 

In [128]:
control_stats = control_data.describe()
control_stats

,Amount Spent,Impressions,Reach,Num Clicks,Num Searches,Num Views,Num Added to Cart,Num Purchases
count,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000
mean,2288.433333,109559.758621,88844.931034,5320.793103,2221.310345,1943.793103,1300.000000,522.793103
std,367.334451,21311.695472,21452.627592,1726.803732,851.025795,764.021907,400.371207,181.810508
min,1757.000000,71274.000000,42859.000000,2277.000000,1001.000000,848.000000,442.000000,222.000000
25%,1945.500000,95191.250000,75300.250000,4122.250000,1629.250000,1249.000000,942.500000,375.500000
50%,2299.500000,112368.000000,91418.000000,5272.396552,2340.000000,1979.500000,1319.500000,506.000000
75%,2532.000000,121259.000000,101958.750000,6609.500000,2655.000000,2360.500000,1638.000000,663.250000
max,3083.000000,145248.000000,127852.000000,8137.000000,4891.000000,4219.000000,1913.000000,800.000000


In [129]:
test_stats = test_data.describe()
test_stats

,Amount Spent,Impressions,Reach,Num Clicks,Num Searches,Num Views,Num Added to Cart,Num Purchases
count,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000
mean,2563.066667,74584.800000,53491.566667,6032.333333,2418.966667,1858.000000,881.533333,521.233333
std,348.687681,32121.377422,28795.775752,1708.567263,388.742312,597.654669,347.584248,211.047745
min,1968.000000,22521.000000,10598.000000,3038.000000,1854.000000,858.000000,278.000000,238.000000
25%,2324.500000,47541.250000,31516.250000,4407.000000,2043.000000,1320.000000,582.500000,298.000000
50%,2584.000000,68853.500000,44219.500000,6242.500000,2395.500000,1881.000000,974.000000,500.000000
75%,2836.250000,99500.000000,78778.750000,7604.750000,2801.250000,2412.000000,1148.500000,701.000000
max,3112.000000,133771.000000,109834.000000,8264.000000,2978.000000,2801.000000,1391.000000,890.000000


## Hypothesis Testing 

To conduct hypothesis testing, I will use SciPy's t-test for one-sided t-test 

My hypotheses are as follows: 

__Null Hypothesis (H<sub>0</sub>)__: There is no difference in purchases between the control campaign and the test campaign

__Alternative Hypothesis (H<sub>A</sub>)__: There is a difference in purchases between the control campaign and the test campaign (other: The test campaign shows a higher amount of purchases)

### Test for Normal Distribution 
Before conducting hypothesis testing, I want to see if the datasets are normally distributed. Because the datasets are smaller in size, I will use the __Shapiro-Wilks Test__. If both are normally distributed, I can continue with the hypothesis test. 

In [130]:
from scipy.stats import shapiro, ttest_ind, mannwhitneyu #ttest_ind will be used if both are normally distributed, 
#mannwhitneyu will be used if one dataset is normally distributed and the other is not 

In [131]:
#1. Extract the columns we want from each dataset: 
purchases_control = control_data['Num Purchases']
purchases_test = test_data['Num Purchases']

#2. Conduct the Shapiro-Wilk test for normality on the control group
stat_control, p_value_control = shapiro(purchases_control)
print('Control Group:')
print('Shapiro-Wilk Test Statistic:', stat_control)
print('p-value:', p_value_control)

# Conducting the Shapiro-Wilk test for normality on the test group
stat_test, p_value_test = shapiro(purchases_test)
print('\nTest Group:')
print('Shapiro-Wilk Test Statistic:', stat_test)
print('p-value:', p_value_test)

# Interpretation of normality test
alpha = 0.05
if p_value_control > alpha:
    print("Control Group: Data is normally distributed")
else:
    print("Control Group: Data is not normally distributed")

if p_value_test > alpha:
    print("Test Group: Data is normally distributed")
else:
    print("Test Group: Data is not normally distributed")


Control Group:
Shapiro-Wilk Test Statistic: 0.9432733058929443
p-value: 0.11144547164440155

Test Group:
Shapiro-Wilk Test Statistic: 0.9181894659996033
p-value: 0.024077769368886948
Control Group: Data is normally distributed
Test Group: Data is not normally distributed


#### Mann Whitney U Test
Because our control group is normally distributed and the test group is not (under 95% confidence), I can use the __Mann Whitney U Test__ to do the hypothesis testing.


In [132]:
u_stat, p_value_u = mannwhitneyu(purchases_test, purchases_control, alternative='greater')
print('\nMann-Whitney U Test:')
print('U-statistic:', u_stat)
print('P-value:', p_value_u)

# Interpretation
if p_value_u < alpha:
    print("Reject the null hypothesis - There are more purchases from the test campaign than in the control campaign")
else:
    print("Fail to reject the null hypothesis - There is no significant difference in purchases between the test and control campaigns")


Mann-Whitney U Test:
U-statistic: 445.0
P-value: 0.5324067103535307
Fail to reject the null hypothesis - There is no significant difference in purchases between the test and control campaigns


Under a 95% confidence interval, we fail to reject the null hypothesis, indicating there is no significant difference in purchases between the test campaign and the control campaign

__What if we use a difference confidence interval?__ <br>
Say, 99%? 

Let's check it out! First, I will have to repeat the test for normality under this confidence (alpha = 0.01), and then base our hypothesis test choice based on that. 

In [133]:
stat_control, p_value_control = shapiro(purchases_control)
print('Control Group:')
print('Shapiro-Wilk Test Statistic:', stat_control)
print('p-value:', p_value_control)

# Conducting the Shapiro-Wilk test for normality on the test group
stat_test, p_value_test = shapiro(purchases_test)
print('\nTest Group:')
print('Shapiro-Wilk Test Statistic:', stat_test)
print('p-value:', p_value_test)

# Interpretation of normality test
alpha = 0.01
if p_value_control > alpha:
    print("Control Group: Data is normally distributed")
else:
    print("Control Group: Data is not normally distributed")

if p_value_test > alpha:
    print("Test Group: Data is normally distributed")
else:
    print("Test Group: Data is not normally distributed")

Control Group:
Shapiro-Wilk Test Statistic: 0.9432733058929443
p-value: 0.11144547164440155

Test Group:
Shapiro-Wilk Test Statistic: 0.9181894659996033
p-value: 0.024077769368886948
Control Group: Data is normally distributed
Test Group: Data is normally distributed


In [134]:
## Under a 99% Confidence Interval, both are normally distributed. Can use ttest_ind from scipy!

# Conduct one-sided t-test
t, p_val = ttest_ind(purchases_test, purchases_control)

#3. Output the t-statistic and p-value
print('T-statistic:', t)
print('P-value:', p_val)

# Interpretation, with 99% confidence (alpha = 0.01)
alpha = 0.01
if p_val < alpha:
    print("Reject the null hypothesis - There are more purchases from the test campaign than in the control campaign")
else:
    print("Fail to reject the null hypothesis - There is no significant difference in purchases between the test and control campaigns")

T-statistic: -0.03066909523750146
P-value: 0.9756387309702421
Fail to reject the null hypothesis - There is no significant difference in purchases between the test and control campaigns


While both of our datasets are normally distributed under the 99% confidence level, our conclusion remains the same- __we fail to reject the null hypothesis, and there is no significant difference in purchases between the test and control campaigns__. 

## Comparing Campaigns Using Visualizations
Hypothesis testing is a critical method to use while conducting A/B testing. Another useful method in comparing the campaigns is through visualizations, as it can make all the numbers clearer to the reader. 

### Feature Engineering 
Before creating visualizations, it would be a good idea to add some metrics to our datasets, and then merge the datasets into one. Some metrics include: 
* __Click-Through Rate (CTR)__: The ratio of of users who click on a link to those who actually view the page, caluclated by Num Clicks / Num Views (times 100 to get a percentage)
* __Cost Per Click (CPC)__: Average cost for each click on an ad/link. calculated by Amount Spent / Num Clicks
* __Conversion Rate (CR)__: Percent of those who clicked a link that actually purchased the product, calculated by Num Purchases/Num Clicks * 100
* __Return on Investment (ROI)__: Profitability of Investment relative to its cost, calculated by revenue generated (Num Purchases) minus the Amount Spent, all divided by the Amount Spent (times 100 to get percentage) 
* __% Unique Impressions__: The percentage of impressions that are unique. Calculated by dividing reach (# unique impressions) divided by Impressions, multiplied by 100. 

To make visualizing easier, I will combine the dataframes, and then add the new features. 

In [321]:
merged_df = pd.concat([control_data, test_data], ignore_index = True)
merged_df.head()

,Campaign,Date,Amount Spent,Impressions,Reach,Num Clicks,Num Searches,Num Views,Num Added to Cart,Num Purchases
0,Control Campaign,2019-01-08,2280,82702.000000,56930.000000,7016.000000,2290.000000,2159.000000,1819.0,618.000000
1,Control Campaign,2019-02-08,1757,121040.000000,102513.000000,8110.000000,2033.000000,1841.000000,1219.0,511.000000
2,Control Campaign,2019-03-08,2343,131711.000000,110862.000000,6508.000000,1737.000000,1549.000000,1134.0,372.000000
3,Control Campaign,2019-04-08,1940,72878.000000,61235.000000,3065.000000,1042.000000,982.000000,1183.0,340.000000
4,Control Campaign,2019-05-08,1835,109559.758621,88844.931034,5320.793103,2221.310345,1943.793103,1300.0,522.793103


In [322]:
merged_df['Click-Through Rate (%)'] = (merged_df['Num Clicks'] / merged_df['Impressions']) * 100 

merged_df['Cost Per Click'] = merged_df['Amount Spent'] / merged_df['Num Clicks']

merged_df['Conversion Rate (%)'] = (merged_df['Num Purchases']/ merged_df['Num Clicks']) * 100 

merged_df['ROI (%)'] = ((merged_df['Num Purchases'] - merged_df['Amount Spent']) / merged_df['Amount Spent']) * 100

merged_df['% Unique Impressions'] = (merged_df['Reach'] / merged_df['Impressions']) * 100

merged_df.head()

,Campaign,Date,Amount Spent,Impressions,Reach,Num Clicks,Num Searches,Num Views,Num Added to Cart,Num Purchases,Click-Through Rate (%),Cost Per Click,Conversion Rate (%),ROI (%),% Unique Impressions
0,Control Campaign,2019-01-08,2280,82702.000000,56930.000000,7016.000000,2290.000000,2159.000000,1819.0,618.000000,8.483471,0.324971,8.808438,-72.894737,68.837513
1,Control Campaign,2019-02-08,1757,121040.000000,102513.000000,8110.000000,2033.000000,1841.000000,1219.0,511.000000,6.700264,0.216646,6.300863,-70.916335,84.693490
2,Control Campaign,2019-03-08,2343,131711.000000,110862.000000,6508.000000,1737.000000,1549.000000,1134.0,372.000000,4.941121,0.360018,5.716042,-84.122919,84.170646
3,Control Campaign,2019-04-08,1940,72878.000000,61235.000000,3065.000000,1042.000000,982.000000,1183.0,340.000000,4.205659,0.632953,11.092985,-82.474227,84.023985
4,Control Campaign,2019-05-08,1835,109559.758621,88844.931034,5320.793103,2221.310345,1943.793103,1300.0,522.793103,4.856521,0.344873,9.825473,-71.509913,81.092668


In [324]:
#merged_df.to_csv('ab_test_analysis_merged.csv')

#### Visualizations 
Some visualizations that we can create are: 
* Time series depicting the purchases
* 

In [137]:
#merged_df.resample(rule='M', on='Date')['Num Purchases'].mean()

#### Get Relevant Data
In our data, we have dates of the 8th of the month for every month in the year. On top of this, we have the data for the majority of the month of August. I will only use the data from the 8th of the month for every month, to make sure that each month has an equal representation. 

In [320]:
eighth_of_month_df = merged_df[merged_df['Date'].isin(['2019-01-08', '2019-02-08','2019-03-08','2019-04-08','2019-05-08','2019-06-08',
                                 '2019-07-08','2019-08-08','2019-09-08','2019-10-08','2019-11-08','2019-12-08'])]

eighth_of_month_df.reset_index(inplace = True)

len(eighth_of_month_df)

24

In [139]:
eighth_of_month_df[eighth_of_month_df['Campaign'] == 'Test Campaign'].describe()

,index,Amount Spent,Impressions,Reach,Num Clicks,Num Searches,Num Views,Num Added to Cart,Num Purchases,Click-Through Rate (%),Cost Per Click,Conversion Rate (%),ROI (%)
count,12.000000,12.00000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000
mean,35.500000,2652.25000,73533.833333,48352.416667,6079.166667,2403.166667,1825.166667,1005.583333,577.000000,9.878464,0.494115,10.419592,-77.842353
std,3.605551,233.04901,30852.350639,28645.363891,1996.457719,379.736950,631.626016,297.423621,217.897808,5.886431,0.209876,5.217351,8.841265
min,30.000000,2297.00000,33669.000000,10598.000000,3038.000000,1854.000000,858.000000,424.000000,255.000000,4.483876,0.299937,3.384615,-91.522606
25%,32.750000,2448.50000,44804.250000,30101.000000,4219.750000,2099.750000,1346.000000,881.250000,408.250000,5.312968,0.338992,7.127054,-85.778078
50%,35.500000,2681.00000,74357.000000,38984.000000,6523.500000,2335.500000,1898.000000,1015.500000,623.000000,7.750055,0.398840,8.486530,-75.258050
75%,38.250000,2832.75000,96470.250000,73372.500000,7945.000000,2766.500000,2237.250000,1247.000000,723.750000,12.802310,0.643425,13.458636,-71.457492
max,41.000000,3008.00000,124591.000000,95138.000000,8264.000000,2899.000000,2761.000000,1382.000000,890.000000,21.337135,0.990125,21.085051,-66.565085


In [140]:
import plotly.graph_objects as go

In [141]:
control = eighth_of_month_df[eighth_of_month_df['Campaign'] == 'Control Campaign']
test = eighth_of_month_df[eighth_of_month_df['Campaign'] == 'Test Campaign']

fig = go.Figure()
# Create and style traces
fig.add_trace(go.Scatter(x= control['Date'], y= control['Num Purchases'], mode='lines', name='Control Campaign',
                         line=dict(color='firebrick', width=4)))
fig.add_trace(go.Scatter(x= test['Date'], y= test['Num Purchases'], name = 'Test Campaign',
                         line=dict(color='royalblue', width=4)))


fig.update_layout(
    title="Number of Purchases Over Time",
    xaxis_title="Date",
    yaxis_title="Number of Purchases",
    legend_title="Campaign Type",
    font=dict(
        family="Overpass",
        size=18,
        color="Black"
    )
)


fig.show()

Insert Analysis Here

In [ ]:
## Basic Metrics 



In [149]:
## Bar Graphs for the %s: CTR, CR, ROI (can do all on one plot) 

metrics =['Click-Through_Rate', 'Conversion Rate', 'Return on Investment']
control_mean_ctr = round(control['Click-Through Rate (%)'].mean(), 2)
control_mean_cr = round(control['Conversion Rate (%)'].mean(), 2)
control_mean_roi = round(control['ROI (%)'].mean(), 2)
control_mean_metrics = [control_mean_ctr, control_mean_cr, control_mean_roi]

test_mean_ctr = round(test['Click-Through Rate (%)'].mean(), 2)
test_mean_cr = round(test['Conversion Rate (%)'].mean(), 2)
test_mean_roi = round(test['ROI (%)'].mean(), 2)
test_mean_metrics = [test_mean_ctr, test_mean_cr, test_mean_roi]

fig = go.Figure(data=[
    go.Bar(name='Control Campaign', x= metrics, y=control_mean_metrics),
    go.Bar(name='Test Campaign', x=metrics, y=test_mean_metrics)
])
# Change the bar mode
#fig.update_layout(barmode='group')
fig.update_layout(
    title="Calculated Campaign Metrics",
    xaxis_title="Metric Type",
    yaxis_title="Mean Metric Value (%)",
    legend_title="Campaign Type",
    font=dict(
        family="Overpass",
        size=18,
        color="Black"
    ), 
    barmode = 'group'
)
fig.show()

Ideas for vizs: <br> 
* Distributions of Impressions, Reach, Num Clicks, Num Searches, Num Views, Num Added to Cart, Num Purchases
* Month of August time series control campaign stats vs test campaign stats

In [155]:
from plotly.subplots import make_subplots

In [261]:
'''Function to create grouped histograms
Parameters- 
* col_name (str): name of column to base the viz on 
* control_mean_annotation_offset (int or float): Where to place the annotation for the control group's 
vertical mean line (dashed) with respect to the vertical line (negative values place the annotation to the left of 
the line, positive values place annotation to the right of the line)
* test_mean_annotation_offset: same as control_mean_annotation offset, except for the test group's vertical mean
line (solid, green line)
* y (int or float): indicates how high or low on the viz the annotations will be 
* bargap (int or float, optional): Adds space between bar groups. Default is 0, or no gap

'''
def make_hist(col_name, control_mean_annotation_offset, test_mean_annotation_offset, y, bargap = 0): 
    x0 = control[col_name]
    x1 = test[col_name]

    fig = go.Figure()
    fig.add_trace(go.Histogram(x=x0, name = 'Control Campaign'))
    fig.add_trace(go.Histogram(x=x1, name = 'Test Campaign'))
    fig.add_vline(x= control[col_name].mean(), line_width=1.5, line_dash="dash", line_color="black")#, annotation_text= 'Control Avg')
    fig.add_annotation(x=control[col_name].mean() + control_mean_annotation_offset, y= y,
                text="Control Avg",
                showarrow=False,
                yshift=10)

    fig.add_vline(x= test[col_name].mean(), line_width=1.5, line_color="green")#, annotation_text= 'Test Avg')
    fig.add_annotation(x=test[col_name].mean() + test_mean_annotation_offset, y=y,
                text="Test Avg",
                showarrow=False,
                yshift=10)

    fig.update_layout(
        title=f"Distribution of {col_name}",
        xaxis_title="Value",
        yaxis_title="Count",
        legend_title="Campaign Type",
        font=dict(
            family="Overpass",
            size=18,
            color="Black"
        ), barmode = 'group', 
    bargap = bargap)
    fig.show()
    
    
#make_graph('Num Purchases', -70, 60, 10)

In [262]:
#impressions
make_hist('Impressions', 10000, -10000, 5, 0.05)

In [263]:
make_hist('Reach', 10000, -9000, 5, 0.05)

In [264]:
make_hist('Num Clicks', -550, 400, 5, 0.05)

In [265]:
make_hist('Num Searches', -200, 150, 10, 0.05)

In [266]:
make_hist('Num Views', -200, 150, 10, 0.05)

In [267]:
make_hist('Num Added to Cart', 120, -100, 10, 0.1)

In [268]:
make_hist('Num Purchases', -60, 50, 8, 0.1)

### Was the amount spent worth it? 

Some ways we can determine this: 



### Month of August
Previously, we were only looking at the values of the 8th of every month. The original data includes the majority of August, from the 8th of the month through the 30th, which we can analyze on a daily basis. 

In [273]:
august_df = merged_df[~merged_df['Date'].isin(['2019-01-08', '2019-02-08','2019-03-08','2019-04-08','2019-05-08','2019-06-08',
                                 '2019-07-08','2019-09-08','2019-10-08','2019-11-08','2019-12-08'])]

august_control = august_df[august_df['Campaign'] == 'Control Campaign']

august_test = august_df[august_df['Campaign'] == 'Test Campaign']


In [276]:
#august_test
#august_control


In [297]:
fig = go.Figure()
# Create and style traces
fig.add_trace(go.Scatter(x= august_control['Date'], y= august_control['Impressions'], mode='lines', name='Control Campaign',
                         line=dict(color='firebrick', width=4)))
fig.add_trace(go.Scatter(x= august_test['Date'], y= august_test['Impressions'], name = 'Test Campaign',
                         line=dict(color='royalblue', width=4)))
fig.add_hline(y= august_control['Impressions'].mean(), line_width=1.5, line_dash="dash", line_color="black") #, annotation_text= 'Control Avg')
fig.add_annotation(x = '2019-08-08', y = august_control['Impressions'].mean() + 5000, 
                   text="Control Avg",showarrow=False)

fig.add_hline(y= august_test['Impressions'].mean(), line_width=1.5, line_color="green") #, annotation_text= 'Test Avg')
fig.add_annotation(x = '2019-08-08', y = august_test['Impressions'].mean() - 5000, 
                   text="Test Avg",showarrow=False)

fig.update_layout(
    title="Number of Impressions Over Time (August 2019)",
    xaxis_title="Date",
    yaxis_title="Number of Impressions",
    legend_title="Campaign Type",
    font=dict(
        family="Overpass",
        size=18,
        color="Black"
    )
)


fig.show()

In [298]:
fig = go.Figure()
# Create and style traces
fig.add_trace(go.Scatter(x= august_control['Date'], y= august_control['Reach'], mode='lines', name='Control Campaign',
                         line=dict(color='firebrick', width=4)))
fig.add_trace(go.Scatter(x= august_test['Date'], y= august_test['Reach'], name = 'Test Campaign',
                         line=dict(color='royalblue', width=4)))
fig.add_hline(y= august_control['Reach'].mean(), line_width=1.5, line_dash="dash", line_color="black") #, annotation_text= 'Control Avg')
fig.add_annotation(x = '2019-08-08', y = august_control['Reach'].mean() + 5000, 
                   text="Control Avg",showarrow=False)

fig.add_hline(y= august_test['Reach'].mean(), line_width=1.5, line_color="green") #, annotation_text= 'Test Avg')
fig.add_annotation(x = '2019-08-08', y = august_test['Reach'].mean() - 5000, 
                   text="Test Avg",showarrow=False)

fig.update_layout(
    title="Reach Over Time (August 2019)",
    xaxis_title="Date",
    yaxis_title="Reach Amount",
    legend_title="Campaign Type",
    font=dict(
        family="Overpass",
        size=18,
        color="Black"
    )
)


fig.show()

In [302]:
fig = go.Figure()
# Create and style traces
fig.add_trace(go.Scatter(x= august_control['Date'], y= august_control['Num Clicks'], mode='lines', name='Control Campaign',
                         line=dict(color='firebrick', width=4)))
fig.add_trace(go.Scatter(x= august_test['Date'], y= august_test['Num Clicks'], name = 'Test Campaign',
                         line=dict(color='royalblue', width=4)))
fig.add_hline(y= august_control['Num Clicks'].mean(), line_width=1.5, line_dash="dash", line_color="black") #, annotation_text= 'Control Avg')
fig.add_annotation(x = '2019-08-08', y = august_control['Num Clicks'].mean() - 450, 
                   text="Control Avg",showarrow=False)

fig.add_hline(y= august_test['Num Clicks'].mean(), line_width=1.5, line_color="green") #, annotation_text= 'Test Avg')
fig.add_annotation(x = '2019-08-08', y = august_test['Num Clicks'].mean() + 450, 
                   text="Test Avg",showarrow=False)

fig.update_layout(
    title="Number of Clicks Over Time (August 2019)",
    xaxis_title="Date",
    yaxis_title="Amount of Clicks",
    legend_title="Campaign Type",
    font=dict(
        family="Overpass",
        size=18,
        color="Black"
    )
)


fig.show()

In [309]:
# Num Searches
fig = go.Figure()
# Create and style traces
fig.add_trace(go.Scatter(x= august_control['Date'], y= august_control['Num Searches'], mode='lines', name='Control Campaign',
                         line=dict(color='firebrick', width=4)))
fig.add_trace(go.Scatter(x= august_test['Date'], y= august_test['Num Searches'], name = 'Test Campaign',
                         line=dict(color='royalblue', width=4)))
fig.add_hline(y= august_control['Num Searches'].mean(), line_width=1.5, line_dash="dash", line_color="black") #, annotation_text= 'Control Avg')
fig.add_annotation(x = '2019-08-08', y = august_control['Num Searches'].mean() - 200, 
                   text="Control Avg",showarrow=False)

fig.add_hline(y= august_test['Num Searches'].mean(), line_width=1.5, line_color="green") #, annotation_text= 'Test Avg')
fig.add_annotation(x = '2019-08-08', y = august_test['Num Searches'].mean() + 200, 
                   text="Test Avg",showarrow=False)

fig.update_layout(
    title="Number of Searches Over Time (August 2019)",
    xaxis_title="Date",
    yaxis_title="Amount of Searches",
    legend_title="Campaign Type",
    font=dict(
        family="Overpass",
        size=18,
        color="Black"
    )
)


fig.show()


In [307]:
#Num Views
fig = go.Figure()
# Create and style traces
fig.add_trace(go.Scatter(x= august_control['Date'], y= august_control['Num Views'], mode='lines', name='Control Campaign',
                         line=dict(color='firebrick', width=4)))
fig.add_trace(go.Scatter(x= august_test['Date'], y= august_test['Num Views'], name = 'Test Campaign',
                         line=dict(color='royalblue', width=4)))
fig.add_hline(y= august_control['Num Views'].mean(), line_width=1.5, line_dash="dash", line_color="black") #, annotation_text= 'Control Avg')
fig.add_annotation(x = '2019-08-08', y = august_control['Num Views'].mean() + 100, 
                   text="Control Avg",showarrow=False)

fig.add_hline(y= august_test['Num Views'].mean(), line_width=1.5, line_color="green") #, annotation_text= 'Test Avg')
fig.add_annotation(x = '2019-08-08', y = august_test['Num Views'].mean() - 100, 
                   text="Test Avg",showarrow=False)

fig.update_layout(
    title="Number of Views Over Time (August 2019)",
    xaxis_title="Date",
    yaxis_title="Amount of Views",
    legend_title="Campaign Type",
    font=dict(
        family="Overpass",
        size=18,
        color="Black"
    )
)


fig.show()



In [312]:
##Num Purchases

fig = go.Figure()
# Create and style traces
fig.add_trace(go.Scatter(x= august_control['Date'], y= august_control['Num Purchases'], mode='lines', name='Control Campaign',
                         line=dict(color='firebrick', width=4)))
fig.add_trace(go.Scatter(x= august_test['Date'], y= august_test['Num Purchases'], name = 'Test Campaign',
                         line=dict(color='royalblue', width=4)))
fig.add_hline(y= august_control['Num Purchases'].mean(), line_width=1.5, line_dash="dash", line_color="black") #, annotation_text= 'Control Avg')
fig.add_annotation(x = '2019-08-11', y = august_control['Num Purchases'].mean() + 30, 
                   text="Control Avg",showarrow=False)

fig.add_hline(y= august_test['Num Purchases'].mean(), line_width=1.5, line_color="green") #, annotation_text= 'Test Avg')
fig.add_annotation(x = '2019-08-11', y = august_test['Num Purchases'].mean() - 30, 
                   text="Test Avg",showarrow=False)

fig.update_layout(
    title="Number of Purchases Over Time (August 2019)",
    xaxis_title="Date",
    yaxis_title="Number of Purchases",
    legend_title="Campaign Type",
    font=dict(
        family="Overpass",
        size=18,
        color="Black"
    )
)


fig.show()

### Results
The results for this ad campaign are relatively straight-forward. Based on the hypothesis testing alone, it is clear that the test campaign did not perform significantly better than the control campaign. Based on the visualizations alone, the control and test campaigns showed similar patterns in metrics over time, and similar distributions. On average, the control campaign out-performed the test campaign across most of the features, with the exception of the number of searches and the number of clicks. 

### Conclusion

#### Pitfalls/What Could Be Done Differently 
With this campaign, there are two main pitfalls that could be changed: <br>
* The __amount of data__, and 
* The __metrics tracked__ <br> 

__AMOUNT OF DATA__: <br>
As with most data-based business problems, the more data (or sample size) we have, the better. In an A/B test, more data can allow for better generalization in results. In the dataset provided, we have a total of 60 samples, with 30 of those for the control campaign, and 30 for the test campaign. Of those 60, 36 are for the majority of the month of August (18 days of August for the control campaign, and the same days for the test campaign). That leaves 24 total samples, containing data from the 8th of the month for each month (12 for the control, 12 for the test; including August 8th). This is very little data. If we wanted to get a better idea of how the campaigns worked, it would be recommended to have data from every single day of the campaign, instead of just one day out of the month and 18 days in August. This is assuming that the original curator of the dataset did not just want to compare the campaigns on the 8th of every month for the year. 


__METRICS TRACKED__: <br> 
While the metrics provided in the dataset do give important insight into how the campaigns faired, there are a few other 

